<a href="https://colab.research.google.com/github/raihanhd12/BIG-DATA/blob/main/MLlib/movie_recommendations_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
os.system("rm -rf /content/ml-predictions.csv")
os.system("rm -rf /content/mymodel")

0

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Read the ratings data from a file
lines = spark.read.text("/content/drive/MyDrive/Kuliah/Semester 6/Big Data/ml-1m/ratings.dat").rdd

# Split each line into fields and create a Row object
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))

# Create a DataFrame from the RDD
ratings = spark.createDataFrame(ratingsRDD)

# Split the data into training and test sets
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()

# Calculate the RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("RMSE: ", rmse)

# Calculate the MSE
mse = predictions.rdd.map(lambda row: (row['rating'] - row['prediction'])**2).mean()
print("MSE: ", mse)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|   148|      1|   5.0|977335193|  4.414916|
|   148|    161|   4.0|977352828| 3.9896765|
|   148|    208|   3.0|979578513| 3.1200483|
|   148|    361|   4.0|979577717|  3.772702|
|   148|    370|   4.0|979578765| 3.7175312|
|   148|    377|   4.0|977333369| 4.1809597|
|   148|    379|   5.0|977333890| 3.7085142|
|   148|    405|   2.0|977334427|  2.603137|
|   148|    419|   3.0|979579310|  3.452846|
|   148|    432|   3.0|979578996| 3.3927321|
|   148|    434|   4.0|977333974|  3.810626|
|   148|    459|   3.0|977334058| 2.6473405|
|   148|    468|   4.0|979577829| 3.3463068|
|   148|    508|   2.0|977352900| 3.8614523|
|   148|    520|   3.0|979579013| 3.8286252|
|   148|    575|   4.0|979578545| 3.8676314|
|   148|    589|   5.0|977333311|  4.277261|
|   148|    594|   4.0|979575755|  4.054224|
|   148|    653|   3.0|977333786|  3.955575|
|   148|  